In [1]:
!pip install pyarrow
!pip install dask
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from datetime import datetime
import pyarrow.parquet as pq
import dask.dataframe as dd
import s3fs
s3 = s3fs.S3FileSystem()

requests 2.20.0 has requirement urllib3<1.25,>=1.21.1, but you'll have urllib3 1.25.8 which is incompatible.
awscli 1.17.5 has requirement botocore==1.14.5, but you'll have botocore 1.14.14 which is incompatible.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
requests 2.20.0 has requirement urllib3<1.25,>=1.21.1, but you'll have urllib3 1.25.8 which is incompatible.
awscli 1.17.5 has requirement botocore==1.14.5, but you'll have botocore 1.14.14 which is incompatible.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

ImportError: numpy.core.multiarray failed to import

In [ ]:
s3_data_path = 's3://datascience-workspace-dev/buyer-intent/consumer-behaviour/training_sets/bi_tmp_union_22aining_set_001_007_030_060_090_20200121'
def load_from_s3_parquet(s3_data_path):
    try:
        df = pq.ParquetDataset(s3_data_path, filesystem=s3).read_pandas().to_pandas()
        return df
    except e as message:
        return None

df = load_from_s3_parquet(s3_data_path)
# Remember you may need bigger machine due to lack of memory

In [ ]:
df.head(10)

In [ ]:
# Select only for sale segment
df = df.loc[ (df['ldp_dominant_segment_for_sale_001'] == 1) | (df['ldp_dominant_segment_for_sale_007'] == 1) | (df['ldp_dominant_segment_for_sale_030'] == 1) | (df['ldp_dominant_segment_for_sale_060'] == 1) | (df['ldp_dominant_segment_for_sale_090'] == 1) ] 
df = df.drop(['member_id_001','member_id_007','member_id_030','member_id_060','member_id_090','snapshot_date_mst_yyyymmdd_001', 
                              'snapshot_date_mst_yyyymmdd_007', 'snapshot_date_mst_yyyymmdd_030', 'snapshot_date_mst_yyyymmdd_060', 'snapshot_date_mst_yyyymmdd_090'], axis = 1)
df_filtered = df.dropna(thresh = 220)
print(df_filtered.shape)

In [ ]:
df.head(10)

In [ ]:
#Sample due to big size of data
df_sampled = df_filtered.sample(frac=0.0005, replace=False, random_state=1)
print(df_sampled.shape)
df_sampled = df_sampled.fillna(0) 
df_sampled.head(30)

In [ ]:
# Save Results for further analysis
df_sampled.to_csv('df_sampled.csv', sep=',', encoding='utf-8')

In [ ]:
df_sampled.describe()

In [ ]:
#Almost There is No correlation among attributes
corr = df_sampled.corr()
corr.style.background_gradient(cmap='coolwarm')